In [1]:
import matplotlib.pyplot as plt
import dlib
import cv2 
import numpy as np
import numpy as np
import time  

In [ ]:
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
dlib_facelanmarks = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


In [ ]:
EAR_THRESHOLD = 0.25  
SLEEP_DURATION = 1.0  
blink_start_time = None

In [ ]:
COLOR_OPEN = (50, 255, 50)   
COLOR_CLOSED = (50, 50, 255) 
COLOR_BG = (40, 40, 40)
COLOR_TEXT = (200, 200, 200)


In [ ]:
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE_SMALL = 0.5
FONT_SCALE_LARGE = 0.7
FONT_THICKNESS = 1


In [ ]:
def eye_aspect_ratio(eye_points):
    eye = np.array(eye_points, dtype=np.float32)
    vertical1 = np.linalg.norm(eye[1] - eye[5])
    vertical2 = np.linalg.norm(eye[2] - eye[4])
    horizontal = np.linalg.norm(eye[0] - eye[3])
    return (vertical1 + vertical2) / (2.0 * horizontal)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    flipped = cv2.flip(frame, 1)
    gray = cv2.cvtColor(flipped, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    current_ear = None

    for face in faces:
        face_landmarks = dlib_facelanmarks(gray, face)

        left_eye = [(face_landmarks.part(n).x, face_landmarks.part(n).y) for n in range(36, 42)]
        right_eye = [(face_landmarks.part(n).x, face_landmarks.part(n).y) for n in range(42, 48)]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        current_ear = (left_ear + right_ear) / 2.0

        eye_color = COLOR_CLOSED if current_ear < EAR_THRESHOLD else COLOR_OPEN
        for (x, y) in left_eye + right_eye:
            cv2.circle(flipped, (x, y), 2, eye_color, -1)

        if current_ear < EAR_THRESHOLD:
            if blink_start_time is None:
                blink_start_time = time.time()
            else:
                closed_duration = time.time() - blink_start_time
                if closed_duration >= SLEEP_DURATION:

                    alert_bg = flipped.copy()
                    cv2.rectangle(alert_bg, (0, 0), (flipped.shape[1], 50), COLOR_CLOSED, -1)
                    alpha = 0.2 * (1 + np.sin(time.time() * 10))
                    cv2.addWeighted(alert_bg, alpha, flipped, 1 - alpha, 0, flipped)
                    cv2.putText(flipped, "SLEEPING!", (flipped.shape[1]//2 - 70, 40), 
                               FONT, 1, (255, 255, 255), 2, cv2.LINE_AA)
        else:
            blink_start_time = None

    cv2.rectangle(flipped, (10, 10), (120, 50), COLOR_BG, -1)
    ear_text = f"EAR: {current_ear:.2f}" if current_ear else "EAR: N/A"
    text_color = COLOR_CLOSED if current_ear and current_ear < EAR_THRESHOLD else COLOR_TEXT
    cv2.putText(flipped, ear_text, (15, 35), FONT, FONT_SCALE_SMALL, 
               text_color, FONT_THICKNESS, cv2.LINE_AA)

    if blink_start_time is not None:
        closed_duration = time.time() - blink_start_time
        progress = min(closed_duration / SLEEP_DURATION, 1.0)
        cv2.rectangle(flipped, (10, 55), (120, 60), COLOR_TEXT, 1)
        cv2.rectangle(flipped, (12, 56), (int(10 + 108 * progress), 59), COLOR_CLOSED, -1)

    cv2.imshow("Eye Monitoring", flipped)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()